# Formats and Meta

## Moochup Course Example:  
```
{
    "id": "0cf46a4b-9022-4cd5-9815-92312b3a84dc",
    "type": "courses",
    "attributes": {
        "name": "KI und Ethik II",
        "courseCode": "KI_Ethik_II",
        "courseMode": "MOOC",
        "abstract": "\Die Entwicklung Künstlicher Intelligenz erfolgt eingebettet in den größeren Kontext einer \u003cstrong\u003edigitalen Transformation\u003c/strong\u003e. Wie wirkt sich [...]",
        "languages": [
            "de"
        ],
        "startDate": "2021-10-18T17:10:00Z",
        "endDate": null,
        "availableUntil": null,
        "image": {
            "url": "https://kicampus-public.s3.openhpicloud.de/courses/orBzqgERBJsLyvTtwv0Oo/visual_v1.png",
            "licenses": [
                {
                    "id": "CC0-1.0",
                    "url": "https://creativecommons.org/publicdomain/zero/1.0",
                    "name": "Creative Commons Zero v1.0 Universal",
                    "author": null
                }
            ]
        },
        "video": null,
        "instructors": [],
        "learningObjectives": [],
        "duration": null,
        "workload": null,
        "partnerInstitute": [],
        "moocProvider": {
            "name": "KI-Campus",
            "url": "https://learn.ki-campus.org/",
            "logo": "https://imgproxy.services.openhpi.de/AcQdY-UlLcD3gY7NcMYcFeVk4-kbZWFH0r31JuyXdI4/fit/0/0/ce/false/aHR0cHM6Ly9sZWFy/bi5raS1jYW1wdXMu/b3JnL2Fzc2V0cy9s/b2dvLTQ3NTE5NjQz/NGEwNjlhNjY1M2Y4/ZDI0MmI2YTU5NDE2/MmEyYTU3MzkxZDky/ZjlkMTFlN2Y0NmMw/M2EyOWU3NWQucG5n.png"
        },
        "courseLicenses": [
            {
                "id": "CC-BY-SA-4.0",
                "url": "https://creativecommons.org/licenses/by-sa/4.0",
                "name": "Creative Commons Attribution Share Alike 4.0 International",
                "author": "KI-Campus"
            }
        ],
        "access": [
            "free"
        ],
        "url": "https://ki-campus.org/courses/KI_Ethik_II"
    }
},
```


## Zammad responses dump (FAQ_Dump.xlsx):
Intent - Antwortvorlagen, manchmal abhängig von Eingabeparametern oder Status eines Kurses:
"wenn B. noch kein Teilnahmebestätigung/Leistungsnachweis (TNB/LN) freigeschaltet: KAM fragen, ob Leistungsnachweis für einen bestimmten Kurs angeboten werden soll/wird  "

Manchmal Prozess definiert unter "Vorgehensweise"


### Datenbeispiel:

Intent: Kosten

Antowortbaustein: 

Liebe/r..,
vielen Dank für Ihre Nachricht - wir freuen uns über Ihr Interesse am KI-Campus!

Kein Sorge, alle Lernangebote auf dem KI-Campus können kostenfrei besucht werden.
Sie haben die Wahl zwischen KI-Campus-Originalen, Partner-Lernangeboten oder externem Material, das von uns sorgsam kuratiert wurde.
Ob es sich bei einem Lernangebot um ein KI-Campus-Original oder ein Partner- oder externes Angebot handelt, erkennen sie an dem auberginefarbenem Button links oben auf dem Kursbild. Im Anhang können Sie passende Beispiele finden.

Wenn sie zusätzlich zum Besuch eines Kurses auch eine Bescheinigung oder ein Zertifikat erlangen wollen, können bei externen Angeboten Kosten entstehen.
Für KI-Campus-Originale gilt das nicht!  Bei uns  sind auch die Teilnahmebestätigungen und Leistungsnachweise kostenlos.

Ich hoffe, ich konnte Ihnen ein wenig weiterhelfen und Sie ermutigen, einfach in unseren Kursen zu stöbern!


## Umsetzung als Assistent:

Textbausteine aus Zammad (1) umschreiben oder (2) prompting dass das LLM diesen Textbaustein zu Beantwortung nutzen soll.

Bei klaren Prozessen evtl. Rasa erweitern und die nötigen API-Calls absetzen - eher eine Ausbaustufe. Ansonsten als Agent?

Wie kann ein LLM Auskunft geben über die Metainformationen? "Z.b. wie viele Kurse habt ihr?"

In [2]:
import requests

API_URL = 'https://learn.ki-campus.org/bridges/moochub/courses'

course_info = []

def fetch_data(url) -> dict:
    response = requests.get(url, headers={'Accept': 'application/vnd.api+json; moochub-version=2.1, application/problem+json'})
    response.raise_for_status()
    return response.json() 

course_info_page = fetch_data(API_URL)
course_info.extend(course_info_page['data'])
while 'next' in course_info_page['links']:
    course_info_page = fetch_data(course_info_page['links']['next'])
    course_info.extend(course_info_page['data'])


HTTPError: 406 Client Error: Not Acceptable for url: https://learn.ki-campus.org/bridges/moochub/courses

In [3]:
print(f"Number of courses {len(course_info)}")
# check if cours id are unique
course_ids = [course['id'] for course in course_info]
print(f"Unique IDs retrieved: {len(course_ids) == len(set(course_ids))}")

print("First course:")
print(course_info[0])

Number of courses 0
Unique IDs retrieved: True
First course:


IndexError: list index out of range

In [4]:
descriptions = [course["attributes"]["description"] for course in course_info]
print("descriptions are html formatted:")
print(descriptions[0])
print(f"Min description length: {min(map(len, descriptions))}")
print(f"Max description length: {max(map(len, descriptions))}")
print(f"Average description length: {sum(map(len, descriptions)) / len(descriptions)}")

abstracts = [course["attributes"]["abstract"] for course in course_info]
abstracts_equal_descriptions = [abstract == description for abstract, description in zip(abstracts, descriptions)]
print(f"Number of abstracts equal to descriptions: {sum(abstracts_equal_descriptions)} of {len(abstracts)}")

names = [course["attributes"]["name"] for course in course_info]
print(f"Min name length: {min(map(len, names))}")
print(f"Max name length: {max(map(len, names))}")
print(f"Average name length: {sum(map(len, names)) / len(names)}")

descriptions are html formatted:


IndexError: list index out of range

In [5]:
# order
descriptions = sorted(descriptions, key=len)
descriptions[-1]

IndexError: list index out of range

In [6]:
print(descriptions[-4])

IndexError: list index out of range

In [7]:
course_languages = ["-".join(course["attributes"]["languages"]) for course in course_info]
print(set(course_languages))

set()


# Course content from Moodle

## Creating a client:
Configure a webservice account with the necessary permissions: https://ki-campus-test.fernuni-hagen.de/admin/settings.php?section=webservicesoverview

In [13]:
import requests
import os
import sys
current = os.path.dirname(os.path.realpath(__file__))
parent = os.path.dirname(current)
sys.path.append(parent)

from loaders.helpers import get_secrets
secrets = get_secrets()
WEBSITE_URL = secrets['DATA_SOURCES']['STAGING']['MOODLE_URL']
TOKEN = secrets['DATA_SOURCES']['STAGING']['MOODLE_TOKEN'] 
BASE_URL = f"https://{WEBSITE_URL}/webservice/rest/server.php"

# create a token through API:
# USERNAME = ""
# PASSWORD = ""
# SERVICESHORTNAME = ""
# print(f"https://{WEBSITE_URL}/login/token.php?username={USERNAME}&password={PASSWORD}&service={SERVICESHORTNAME}")
# !curl "https://{WEBSITE_URL}/login/token.php?username={USERNAME}&password={PASSWORD}&service={SERVICESHORTNAME}"


NameError: name '__file__' is not defined

In [14]:
# example call
function = "core_course_get_categories"
print(BASE_URL)
!curl "{WEBSITE_URL}/webservice/rest/server.php?wstoken={TOKEN}&wsfunction={function}&moodlewsrestformat=json"

NameError: name 'BASE_URL' is not defined

In [15]:
def get_data(fuction: str, **kwargs) -> dict:
    params = {
            "wstoken": TOKEN,
            "wsfunction": fuction,
            "moodlewsrestformat": "json",
        } 
    params.update(kwargs)

    response = requests.get(url=BASE_URL, params=params)
    return response.json()

In [16]:
# overview of all courses
get_data("core_course_get_courses")

NameError: name 'TOKEN' is not defined

In [17]:
# Overview of all course contents ("Modules"/"Abschnitte"/"Topics" and their sub "modules" containing multiple different formats)
# + course description (differently from other API?)
# + Topic summary
# + content for some for files (images, hp5, hmtl) there is a link provided, but for questionaire?

get_data("core_course_get_contents", courseid=16) # Dr. med. KI - Basler Modul https://ki-campus-test.fernuni-hagen.de/course/view.php?id=16


NameError: name 'TOKEN' is not defined

In [18]:
# Html content does not match what I see at https://ki-campus-test.fernuni-hagen.de/course/view.php?id=16 for "Was ist Intelligenz" (where it is referenced) 
# - but "Data Bias" in Module 4, where the link is also referenced 
# How to work with h5p content?
params = {"token": TOKEN,} 
response = requests.get(url="https://ki-campus-test.fernuni-hagen.de/webservice/pluginfile.php/403/mod_page/content/index.html?forcedownload=1", params=params)
print(response.content)

NameError: name 'TOKEN' is not defined

In [19]:
# different type of content items, does not seem relevant for now
get_data("core_course_get_course_content_items", courseid=16) # Dr. med. KI - Basler Modul https://ki-campus-test.fernuni-hagen.de/course/view.php?id=16


NameError: name 'TOKEN' is not defined

In [20]:
# Does not contain any content, only the titel seems no like no interesting new infromation compared to core_course_get_contents
get_data("core_course_get_course_module", cmid=162) # Dr. med. KI - Basler Modul umfrage https://ki-campus-test.fernuni-hagen.de/mod/questionnaire/view.php?id=161


NameError: name 'TOKEN' is not defined

In [21]:
# Enthält nur den Titel des Inhalts, in anderem Format
html_content = get_data("core_course_get_module", id=162) # Dr. med. KI - Basler Modul umfrage https://ki-campus-test.fernuni-hagen.de/mod/questionnaire/view.php?id=161
print(html_content)


NameError: name 'TOKEN' is not defined

In [22]:
# core_course_get_courses_by_field
html_content = get_data("core_course_get_module", id=162) # Dr. med. KI - Basler Modul umfrage https://ki-campus-test.fernuni-hagen.de/mod/questionnaire/view.php?id=161
print(html_content)

NameError: name 'TOKEN' is not defined

# Is the moochup API a subset of moodle API 

In [23]:
from src.loaders.moochup import fetch_data
from src.loaders.moodle import Moodle

moodle = Moodle(environment="PRODUCTION").extract()
moodle

ImportError: cannot import name 'fetch_data' from 'src.loaders.moochup' (C:\Users\FatemehSalehi\Documents\kic-web-assistant\src\loaders\moochup.py)

In [24]:
moochup = fetch_data("https://moodle.ki-campus.org/local/open_api/courses.php")
moochup

{'links': {'self': 'https://moodle.ki-campus.org/local/open_api/courses.php?page=0',
  'first': 'https://moodle.ki-campus.org/local/open_api/courses.php?page=0',
  'last': 'https://moodle.ki-campus.org/local/open_api/courses.php?page=0',
  'next': '',
  'previous': ''},
 'data': [{'id': '1',
   'type': 'courses',
   'attributes': {'name': 'KI-Campus',
    'abstract': '<p dir="ltr" style="text-align: left;">Hier findest du eine Übersicht Deiner eingeschriebenen Kurse im KI-Campus-Moodle. Eine Übersicht über alle Lernangebote des KI-Campus findest du auf unserer Hauptseite. <br></p>\r\n<p dir="ltr" style="text-align: left;"><br></p>\r\n<div class="navitem" style="text-align: center;"><a class="btn btn-primary" href="https://ki-campus.org/overview">Zurück zu allen Lernangeboten</a></div>',
    'languages': [],
    'uniformResourceLocator': 'https://moodle.ki-campus.org/course/view.php?id=1',
    'instructors': [{'id': '11450', 'name': 'Jasmina Idler'},
     {'id': '3251', 'name': 'Natalie

In [25]:
# test if the identical courses are in both sources
moodle_ids = [course.id for course in moodle]
moochup_ids = [int(course.id) for course in moochup]
print(f"Number of courses in moodle but not in moochup: {len(set(moodle_ids) - set(moochup_ids))}")
print(f"Number of courses in moochup but not in moodle: {len(set(moochup_ids) - set(moodle_ids))}")

NameError: name 'moodle' is not defined

In [26]:
# test if names are identical
joined_courses = [(moodle_course, moochup_course) for moodle_course in moodle for moochup_course in moochup if moodle_course.id == int(moochup_course.id)]
for course in joined_courses:
    print(f"{course[0].shortname} - {course[0].fullname} - {course[1].attributes.name}")

NameError: name 'moodle' is not defined

In [27]:
# test if content is identical
for course in joined_courses:
    print(course[0].summary == course[1].attributes.abstract)
    if not course[0].summary == course[1].attributes.abstract:
        print(f"{course[0].summary} - {course[1].attributes.abstract}")
        print("-"*100)
    

NameError: name 'joined_courses' is not defined